In [1]:
%matplotlib widget

import pdb
from tqdm import tqdm

import numpy as np
import pandas as pd

from numpy.random import shuffle, choice

from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec

from sklearn.decomposition import PCA

from keras.models import Sequential
from keras.layers import Dense, Activation

/home/proprietaire/Envs/GeomAbs/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
Using TensorFlow backend.


In [2]:
import os, sys
lib_path = os.path.abspath('../methods')
sys.path.insert(0, lib_path)

from models import Model
from data_tools import ImageDataset
import data_tools as dt

In [3]:
n_train = 1350
n_test = 450

(x_train, y_train), (x_test, y_test) = dt.generate_nclasses_dataset(
    nb_samples=n_train+n_test,
    nb_classes=18,
    width_in_pixels=56,
    noise=60,
    free_location=True,
    verbose=False,
    nb_test_samples=n_test)

Class 0 is a rectangle


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Class 1 is a triangle


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Class 2 is a disk


...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Class 16 is a triangle


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Class 17 is a rectangle


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
shapes_18 = ImageDataset(x_train, y_train, x_test, y_test)

In [5]:
def display_classes(ds):
    fig = plt.figure(figsize=(5, 5*ds.n_labels))
    axes = []
    gs1 = gridspec.GridSpec(ds.n_labels,1)
    gs1.update(wspace=0.025, hspace=0.2)
    
    for lbl in range(ds.n_labels):
        filt_lbl = np.nonzero(np.isin(ds.train['y'], [lbl]))[0]
        lbl_spl_id = choice(filt_lbl, size=1)
        lbl_spl = ds.train['x'][lbl_spl_id,:]
    
        ax = plt.subplot(gs1[lbl])
        ax.imshow(
				lbl_spl.reshape(ds.axes_dim)
			)

In [6]:
display_classes(shapes_18)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
w_in = shapes_18.tot_dim
w_1 = 100
w_2 = 100
max_epochs = 400

In [8]:
# Currently dichotomies will only be binary
shapes_parity = [list(map(lambda x: 2*x, range(9))), list(map(lambda x: 2*x + 1, range(9)))]
shapes_smallness = [range(0,9), range(9,18)]

In [9]:
shapes_18.build_dichLabels(shapes_smallness, 'smaller_than_9')
shapes_18.build_dichLabels(shapes_parity, 'parity')

shapes_18.hstack_dichs('parity', 'smaller_than_9')

## 2) Studying the effect of supervision format for representation in hidden layers

### 2.1) Horizontally-stacked supervision

In [10]:
dnn_hstack = Sequential([
    Dense(w_1, input_shape=(w_in,)),
    Activation('tanh'),
    Dense(w_2),
    Activation('tanh'),
    Dense(4),
    Activation('tanh')
])

model_hstack = Model(dnn_hstack)

# Train the model, iterating on the data in batches of 32 samples
model_hstack.fit(shapes_18, dich_name='parity_hstack_smaller_than_9', epochs=10, batch_size=32)

model_hstack.evaluate(shapes_18, dich_name='parity_hstack_smaller_than_9', batch_size=128)

{'test_loss': 0.0027700039765073195, 'test_accuracy': 0.4511111080646515}

In [11]:
model_hstack.sample_eval(shapes_18, 2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
pca2 = PCA(n_components=2)

reduced_hstack_2d = model_hstack.get_repr(shapes_18, shapes_18.spl, pca2)
fig1 = model_hstack.plot_reprs(shapes_18, shapes_18.spl, pca2)

fig1.canvas.layout.width = '80%'
fig1.canvas.layout.height = '5000px'

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
for lay_id, rpr in enumerate(reduced_hstack_2d):
    print('Layer {0:d} - {1:.1f}% 2d var - {2:.1f}% + {3:.1f}%'.format(lay_id, 100*(rpr['reduced']['expl_var'][0]+rpr['reduced']['expl_var'][1]), 100*rpr['reduced']['expl_var'][0], 100*rpr['reduced']['expl_var'][1]))

Layer 0 - 75.3% 2d var - 59.8% + 15.5%
Layer 1 - 78.4% 2d var - 49.5% + 28.8%
Layer 2 - 89.4% 2d var - 60.9% + 28.5%
Layer 3 - 85.1% 2d var - 60.3% + 24.8%
Layer 4 - 99.9% 2d var - 71.4% + 28.5%
Layer 5 - 100.0% 2d var - 69.7% + 30.3%


In [14]:
pca3 = PCA(n_components=3)

reduced_hstack_3d = model_hstack.get_repr(shapes_18, shapes_18.spl, pca3)
fig2 = model_hstack.plot_reprs(shapes_18, shapes_18.spl, pca3)

fig2.canvas.layout.width = '80%'
fig2.canvas.layout.height = '5000px'

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
for lay_id, rpr in enumerate(reduced_hstack_3d):
    print('Layer {0:d} - {1:.1f}% 3d var - {2:.1f}% + {3:.1f}% + {4:.1f}%'.format(lay_id, 100*(rpr['reduced']['expl_var'][0]+rpr['reduced']['expl_var'][1]+rpr['reduced']['expl_var'][2]), 100*rpr['reduced']['expl_var'][0], 100*rpr['reduced']['expl_var'][1], 100*rpr['reduced']['expl_var'][2]))

Layer 0 - 87.9% 3d var - 59.8% + 15.5% + 12.6%
Layer 1 - 95.0% 3d var - 49.5% + 28.8% + 16.6%
Layer 2 - 98.9% 3d var - 60.9% + 28.5% + 9.5%
Layer 3 - 97.2% 3d var - 60.3% + 24.8% + 12.1%
Layer 4 - 100.0% 3d var - 71.4% + 28.5% + 0.1%
Layer 5 - 100.0% 3d var - 69.7% + 30.3% + 0.0%


### 2.2) Product supervision

In [16]:
shapes_prod = [list(set(s1).intersection(set(s2))) for s2 in shapes_smallness for s1 in shapes_parity]
shapes_18.build_dichLabels(shapes_prod, 'parity_prod_smaller_than_9')

In [19]:
dnn_prod = Sequential([
    Dense(w_1, input_shape=(w_in,)),
    Activation('tanh'),
    Dense(w_2),
    Activation('tanh'),
    Dense(4),
    Activation('tanh')
])

model = Model(dnn_prod)

# Train the model, iterating on the data in batches of 32 samples
model.fit(shapes_18, dich_name='parity_prod_smaller_than_9', epochs=10, batch_size=32)

model.evaluate(shapes_18, dich_name='parity_prod_smaller_than_9', batch_size=128)

{'test_loss': 0.0011189340313689576, 'test_accuracy': 1.0}

In [20]:
model.sample_eval(shapes_18, 2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
pca2 = PCA(n_components=2)

reduced_prod_2d = model.get_repr(shapes_18, shapes_18.spl, pca2)
fig3 = model.plot_reprs(shapes_18, shapes_18.spl, pca2)

fig3.canvas.layout.width = '80%'
fig3.canvas.layout.height = '5000px'

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
for lay_id, rpr in enumerate(reduced_prod_2d):
    print('Layer {0:d} - {1:.1f}% 2d var - {2:.1f}% + {3:.1f}%'.format(lay_id, 100*(rpr['reduced']['expl_var'][0]+rpr['reduced']['expl_var'][1]), 100*rpr['reduced']['expl_var'][0], 100*rpr['reduced']['expl_var'][1]))

Layer 0 - 75.0% 2d var - 52.7% + 22.3%
Layer 1 - 77.7% 2d var - 45.5% + 32.2%
Layer 2 - 82.4% 2d var - 44.5% + 37.8%
Layer 3 - 81.6% 2d var - 43.2% + 38.4%
Layer 4 - 81.8% 2d var - 46.7% + 35.1%
Layer 5 - 81.7% 2d var - 50.8% + 30.8%


In [23]:
pca3 = PCA(n_components=3)

reduced_prod_3d = model.get_repr(shapes_18, shapes_18.spl, pca3)
fig4 = model.plot_reprs(shapes_18, shapes_18.spl, pca3)

fig4.canvas.layout.width = '80%'
fig4.canvas.layout.height = '5000px'

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
for lay_id, rpr in enumerate(reduced_prod_3d):
    print('Layer {0:d} - {1:.1f}% 3d var - {2:.1f}% + {3:.1f}% + {4:.1f}%'.format(lay_id, 100*(rpr['reduced']['expl_var'][0]+rpr['reduced']['expl_var'][1]+rpr['reduced']['expl_var'][2]), 100*rpr['reduced']['expl_var'][0], 100*rpr['reduced']['expl_var'][1], 100*rpr['reduced']['expl_var'][2]))

Layer 0 - 88.3% 3d var - 52.7% + 22.3% + 13.4%
Layer 1 - 95.3% 3d var - 45.5% + 32.2% + 17.6%
Layer 2 - 99.7% 3d var - 44.5% + 37.8% + 17.4%
Layer 3 - 99.9% 3d var - 43.2% + 38.4% + 18.3%
Layer 4 - 100.0% 3d var - 46.7% + 35.1% + 18.2%
Layer 5 - 100.0% 3d var - 50.8% + 30.8% + 18.3%


### 2.3) Compact hstacking supervision

In [25]:
shapes_18.compstack_dichs('parity', 'smaller_than_9')

In [26]:
dnn_compstack = Sequential([
    Dense(w_1, input_shape=(w_in,)),
    Activation('tanh'),
    Dense(w_2),
    Activation('tanh'),
    Dense(2),
    Activation('tanh')
])

compstack = Model(dnn_compstack)

# Train the model, iterating on the data in batches of 32 samples
compstack.fit(shapes_18, dich_name='parity_compstack_smaller_than_9', epochs=10, batch_size=32)

compstack.evaluate(shapes_18, dich_name='parity_compstack_smaller_than_9', batch_size=128)

{'test_loss': 0.0009965675065500869, 'test_accuracy': 0.46000000834465027}

In [30]:
pca2 = PCA(n_components=2)

reduced_compstack_2d = compstack.get_repr(shapes_18, shapes_18.spl, pca2)
fig5 = compstack.plot_reprs(shapes_18, shapes_18.spl, pca2)

fig5.canvas.layout.width = '80%'
fig5.canvas.layout.height = '5000px'

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [28]:
for lay_id, rpr in enumerate(reduced_compstack_2d):
    print('Layer {0:d} - {1:.1f}% 2d var - {2:.1f}% + {3:.1f}%'.format(lay_id, 100*(rpr['reduced']['expl_var'][0]+rpr['reduced']['expl_var'][1]), 100*rpr['reduced']['expl_var'][0], 100*rpr['reduced']['expl_var'][1]))

Layer 0 - 77.1% 2d var - 65.3% + 11.8%
Layer 1 - 77.9% 2d var - 50.9% + 27.0%
Layer 2 - 94.7% 2d var - 68.3% + 26.4%
Layer 3 - 89.2% 2d var - 57.4% + 31.8%
Layer 4 - 100.0% 2d var - 69.5% + 30.5%
Layer 5 - 100.0% 2d var - 68.8% + 31.2%


In [29]:
pca3 = PCA(n_components=3)

reduced_compstack_3d = compstack.get_repr(shapes_18, shapes_18.spl, pca3)
fig6 = compstack.plot_reprs(shapes_18, shapes_18.spl, pca3)

fig6.canvas.layout.width = '80%'
fig6.canvas.layout.height = '5000px'

ValueError: could not broadcast input array from shape (2) into shape (3)

In [ ]:
for lay_id, rpr in enumerate(reduced_compstack_3d):
    print('Layer {0:d} - {1:.1f}% 3d var - {2:.1f}% + {3:.1f}% + {4:.1f}%'.format(lay_id, 100*(rpr['reduced']['expl_var'][0]+rpr['reduced']['expl_var'][1]+rpr['reduced']['expl_var'][2]), 100*rpr['reduced']['expl_var'][0], 100*rpr['reduced']['expl_var'][1], 100*rpr['reduced']['expl_var'][2]))

## 3) Testing representation for categorical features

### 3.1) hstack

In [31]:
# Currently dichotomies will only be binary
shapes_range = [range(0,6), range(6,12), range(12,18)]

In [32]:
shapes_18.build_dichLabels(shapes_range, 'range_bins')

shapes_18.hstack_dichs('parity', 'range_bins')
shapes_18.compstack_dichs('parity', 'range_bins')

shapes_18.build_catLabels(shapes_range, 'cat_range_bins')
shapes_18.compstack_dichs('parity', 'cat_range_bins')

In [33]:
dnn_ctg_hstack = Sequential([
    Dense(w_1, input_shape=(w_in,)),
    Activation('tanh'),
    Dense(w_2),
    Activation('tanh'),
    Dense(5),
    Activation('tanh')
])

ctg_hstack = Model(dnn_ctg_hstack)

# Train the model, iterating on the data in batches of 32 samples
ctg_hstack.fit(shapes_18, dich_name='parity_hstack_range_bins', epochs=20, batch_size=32)

ctg_hstack.evaluate(shapes_18, dich_name='parity_hstack_range_bins', batch_size=128)

{'test_loss': 0.0010979679796016878, 'test_accuracy': 0.4955555498600006}

In [34]:
pca2 = PCA(n_components=2)

reduced_ctg_hstack_2d = ctg_hstack.get_repr(shapes_18, shapes_18.spl, pca2)
fig7 = ctg_hstack.plot_reprs(shapes_18, shapes_18.spl, pca2)

fig7.canvas.layout.width = '80%'
fig7.canvas.layout.height = '5000px'

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [35]:
for lay_id, rpr in enumerate(reduced_ctg_hstack_2d):
    print('Layer {0:d} - {1:.1f}% 2d var - {2:.1f}% + {3:.1f}%'.format(lay_id, 100*(rpr['reduced']['expl_var'][0]+rpr['reduced']['expl_var'][1]), 100*rpr['reduced']['expl_var'][0], 100*rpr['reduced']['expl_var'][1]))

Layer 0 - 67.1% 2d var - 50.1% + 17.0%
Layer 1 - 52.2% 2d var - 30.6% + 21.6%
Layer 2 - 68.6% 2d var - 40.4% + 28.1%
Layer 3 - 66.0% 2d var - 40.9% + 25.1%
Layer 4 - 78.2% 2d var - 50.6% + 27.7%
Layer 5 - 78.3% 2d var - 49.2% + 29.1%


In [36]:
pca3 = PCA(n_components=3)

reduced_ctg_hstack_3d = ctg_hstack.get_repr(shapes_18, shapes_18.spl, pca3)
fig8 = ctg_hstack.plot_reprs(shapes_18, shapes_18.spl, pca3)

fig8.canvas.layout.width = '80%'
fig8.canvas.layout.height = '5000px'

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [37]:
for lay_id, rpr in enumerate(reduced_ctg_hstack_3d):
    print('Layer {0:d} - {1:.1f}% 3d var - {2:.1f}% + {3:.1f}% + {4:.1f}%'.format(lay_id, 100*(rpr['reduced']['expl_var'][0]+rpr['reduced']['expl_var'][1]+rpr['reduced']['expl_var'][2]), 100*rpr['reduced']['expl_var'][0], 100*rpr['reduced']['expl_var'][1], 100*rpr['reduced']['expl_var'][2]))

Layer 0 - 80.5% 3d var - 50.1% + 17.0% + 13.5%
Layer 1 - 71.0% 3d var - 30.6% + 21.6% + 18.8%
Layer 2 - 89.2% 3d var - 40.4% + 28.1% + 20.7%
Layer 3 - 88.3% 3d var - 40.9% + 25.1% + 22.4%
Layer 4 - 99.8% 3d var - 50.6% + 27.7% + 21.6%
Layer 5 - 100.0% 3d var - 49.2% + 29.1% + 21.7%


### 3.2) Product

In [38]:
shapes_ctg_prod = [set(s1).intersection(set(s2)) for s2 in shapes_range for s1 in shapes_parity]
shapes_18.build_dichLabels(shapes_ctg_prod, 'parity_prod_range_bins')

In [39]:
w_in =shapes_18.tot_dim
w_1 = 100
w_2 = 100
max_epochs = 400

In [40]:
dnn_ctg_prod = Sequential([
    Dense(w_1, input_shape=(w_in,)),
    Activation('tanh'),
    Dense(w_2),
    Activation('tanh'),
    Dense(6),
    Activation('tanh')
])

ctg_prod = Model(dnn_ctg_prod)

# Train the model, iterating on the data in batches of 32 samples
ctg_prod.fit(shapes_18, dich_name='parity_prod_range_bins', epochs=10, batch_size=32)

ctg_prod.evaluate(shapes_18, dich_name='parity_prod_range_bins', batch_size=128)

{'test_loss': 0.001529807080514729, 'test_accuracy': 1.0}

In [41]:
pca2 = PCA(n_components=2)

reduced_ctg_prod_2d = ctg_prod.get_repr(shapes_18, shapes_18.spl, pca2)
fig9 = ctg_prod.plot_reprs(shapes_18, shapes_18.spl, pca2)

fig9.canvas.layout.width = '80%'
fig9.canvas.layout.height = '5000px'

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [42]:
for lay_id, rpr in enumerate(reduced_ctg_prod_2d):
    print('Layer {0:d} - {1:.1f}% 2d var - {2:.1f}% + {3:.1f}%'.format(lay_id, 100*(rpr['reduced']['expl_var'][0]+rpr['reduced']['expl_var'][1]), 100*rpr['reduced']['expl_var'][0], 100*rpr['reduced']['expl_var'][1]))

Layer 0 - 65.2% 2d var - 47.0% + 18.2%
Layer 1 - 50.0% 2d var - 26.8% + 23.3%
Layer 2 - 57.3% 2d var - 33.0% + 24.3%
Layer 3 - 50.0% 2d var - 27.9% + 22.1%
Layer 4 - 55.8% 2d var - 31.5% + 24.2%
Layer 5 - 56.0% 2d var - 30.2% + 25.8%


In [43]:
pca3 = PCA(n_components=3)

reduced_ctg_prod_3d = ctg_prod.get_repr(shapes_18, shapes_18.spl, pca3)
fig8 = ctg_prod.plot_reprs(shapes_18, shapes_18.spl, pca3)

fig8.canvas.layout.width = '80%'
fig8.canvas.layout.height = '5000px'

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [44]:
for lay_id, rpr in enumerate(reduced_ctg_prod_3d):
    print('Layer {0:d} - {1:.1f}% 3d var - {2:.1f}% + {3:.1f}% + {4:.1f}%'.format(lay_id, 100*(rpr['reduced']['expl_var'][0]+rpr['reduced']['expl_var'][1]+rpr['reduced']['expl_var'][2]), 100*rpr['reduced']['expl_var'][0], 100*rpr['reduced']['expl_var'][1], 100*rpr['reduced']['expl_var'][2]))

Layer 0 - 77.2% 3d var - 47.0% + 18.2% + 12.0%
Layer 1 - 68.4% 3d var - 26.8% + 23.3% + 18.4%
Layer 2 - 76.7% 3d var - 33.0% + 24.3% + 19.4%
Layer 3 - 70.4% 3d var - 27.9% + 22.1% + 20.4%
Layer 4 - 74.2% 3d var - 31.5% + 24.2% + 18.4%
Layer 5 - 74.9% 3d var - 30.2% + 25.8% + 18.9%


### 3.3) compstack

In [45]:
w_in =shapes_18.tot_dim
w_1 = 100
w_2 = 100
max_epochs = 400

In [50]:
dnn_ctg_compstack = Sequential([
    Dense(w_1, input_shape=(w_in,)),
    Activation('tanh'),
    Dense(w_2),
    Activation('tanh'),
    Dense(2)
])

ctg_compstack = Model(dnn_ctg_compstack)

# Train the model, iterating on the data in batches of 32 samples
ctg_compstack.fit(shapes_18, dich_name='parity_compstack_cat_range_bins', epochs=10, batch_size=32)

ctg_compstack.evaluate(shapes_18, dich_name='parity_compstack_cat_range_bins', batch_size=128)

{'test_loss': 0.19431929455863106, 'test_accuracy': 0.8533333539962769}

In [53]:
pca2 = PCA(n_components=2)

reduced_ctg_compstack_2d = ctg_compstack.get_repr(shapes_18, shapes_18.spl, pca2)
fig11 = ctg_compstack.plot_reprs(shapes_18, shapes_18.spl, pca2)

fig11.canvas.layout.width = '80%'
fig11.canvas.layout.height = '5000px'

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [54]:
for lay_id, rpr in enumerate(reduced_ctg_compstack_2d):
    print('Layer {0:d} - {1:.1f}% 2d var - {2:.1f}% + {3:.1f}%'.format(lay_id, 100*(rpr['reduced']['expl_var'][0]+rpr['reduced']['expl_var'][1]), 100*rpr['reduced']['expl_var'][0], 100*rpr['reduced']['expl_var'][1]))

Layer 0 - 65.2% 2d var - 48.7% + 16.4%
Layer 1 - 54.3% 2d var - 31.2% + 23.1%
Layer 2 - 75.3% 2d var - 53.6% + 21.6%
Layer 3 - 77.2% 2d var - 52.6% + 24.6%
Layer 4 - 100.0% 2d var - 77.1% + 22.9%


In [55]:
pca3 = PCA(n_components=3)

reduced_ctg_compstack_3d = ctg_compstack.get_repr(shapes_18, shapes_18.spl, pca3)
fig12 = ctg_compstack.plot_reprs(shapes_18, shapes_18.spl, pca3)

fig12.canvas.layout.width = '80%'
fig12.canvas.layout.height = '5000px'

ValueError: could not broadcast input array from shape (2) into shape (3)

In [56]:
for lay_id, rpr in enumerate(reduced_ctg_compstack_3d):
    print('Layer {0:d} - {1:.1f}% 3d var - {2:.1f}% + {3:.1f}% + {4:.1f}%'.format(lay_id, 100*(rpr['reduced']['expl_var'][0]+rpr['reduced']['expl_var'][1]+rpr['reduced']['expl_var'][2]), 100*rpr['reduced']['expl_var'][0], 100*rpr['reduced']['expl_var'][1], 100*rpr['reduced']['expl_var'][2]))

NameError: name 'reduced_ctg_compstack_3d' is not defined